# 実践演習 Day 1：streamlitとFastAPIのデモ
このノートブックでは以下の内容を学習します。

- 必要なライブラリのインストールと環境設定
- Hugging Faceからモデルを用いたStreamlitのデモアプリ
- FastAPIとngrokを使用したAPIの公開方法

演習を始める前に、HuggingFaceとngrokのアカウントを作成し、
それぞれのAPIトークンを取得する必要があります。


演習の時間では、以下の3つのディレクトリを順に説明します。

1. 01_streamlit_UI
2. 02_streamlit_app
3. 03_FastAPI

2つ目や3つ目からでも始められる様にノートブックを作成しています。

復習の際にもこのノートブックを役立てていただければと思います。

### 注意事項
「02_streamlit_app」と「03_FastAPI」では、GPUを使用します。

これらを実行する際は、Google Colab画面上のメニューから「編集」→ 「ノートブックの設定」

「ハードウェアアクセラレーター」の項目の中から、「T4 GPU」を選択してください。

このノートブックのデフォルトは「CPU」になっています。

---

# 環境変数の設定（1~3共有）


必要なAPIトークンを.envに設定します。

「lecture-ai-engineering/day1」の配下に、「.env_template」ファイルが存在しています。

隠しファイルのため表示されていない場合は、画面左側のある、目のアイコンの「隠しファイルの表示」ボタンを押してください。

「.env_template」のファイル名を「.env」に変更します。「.env」ファイルを開くと、以下のような中身になっています。


```
HUGGINGFACE_TOKEN="hf-********"
NGROK_TOKEN="********"
```
ダブルクオーテーションで囲まれた文字列をHuggingfaceのアクセストークンと、ngrokの認証トークンで書き変えてください。

それぞれのアカウントが作成済みであれば、以下のURLからそれぞれのトークンを取得できます。

- Huggingfaceのアクセストークン
https://huggingface.co/docs/hub/security-tokens

- ngrokの認証トークン
https://dashboard.ngrok.com/get-started/your-authtoken

書き換えたら、「.env」ファイルをローカルのPCにダウンロードしてください。

「01_streamlit_UI」から「02_streamlit_app」へ進む際に、CPUからGPUの利用に切り替えるため、セッションが一度切れてしまいます。

その際に、トークンを設定した「.env」ファイルは再作成することになるので、その手間を減らすために「.env」ファイルをダウンロードしておくと良いです。

「.env」ファイルを読み込み、環境変数として設定します。次のセルを実行し、最終的に「True」が表示されていればうまく読み込めています。

In [1]:
%cd .

c:\Users\ks130\Desktop\aie_lecture\lecture-ai-engineering\day1_local


In [2]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv("./.env")

True

In [3]:
os.environ["NGROK_TOKEN"]

'2vnbSe1xwQKWIYMe1gyGglh5BGo_7vthZfqAint7TjfBWTTVP'

# 01_streamlit_UI

ngrokのトークンを使用して、認証を行います。

In [4]:
from pyngrok import conf
import os
conf.get_default().auth_token = os.environ["NGROK_TOKEN"]

アプリを起動します。

In [6]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!uv run streamlit run 01_streamlit_UI/app.py

公開URL: https://9236-2400-2410-2da2-2300-a5a0-6f-b1e9-469.ngrok-free.app
^C



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://192.168.3.14:8502



- ローカルで実行する場合は、ngrokを使う必要はなく、ターミナルで`uv run streamlit run day1_local/01_streamlit_UI/app.py`を実行

公開URLの後に記載されているURLにブラウザでアクセスすると、streamlitのUIが表示されます。

app.pyのコメントアウトされている箇所を編集することで、UIがどの様に変化するか確認してみましょう。

streamlitの公式ページには、ギャラリーページがあります。

streamlitを使うとpythonという一つの言語であっても、様々なUIを実現できることがわかると思います。

https://streamlit.io/gallery

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [7]:
from pyngrok import ngrok
ngrok.kill()

# 02_streamlit_app

In [8]:
from huggingface_hub import login
login()

stramlitでHuggingfaceのトークン情報を扱うために、streamlit用の設定ファイル（.streamlit）を作成し、トークンの情報を格納します。

In [9]:
# .streamlit/secrets.toml ファイルを作成
import os
import toml

# 設定ファイルのディレクトリ確保
os.makedirs('.streamlit', exist_ok=True)

# 環境変数から取得したトークンを設定ファイルに書き込む
secrets = {
    "huggingface": {
        "token": os.environ.get("HUGGINGFACE_TOKEN", "")
    }
}

# 設定ファイルを書き込む
with open('.streamlit/secrets.toml', 'w') as f:
    toml.dump(secrets, f)

アプリを起動します。

02_streamlit_appでは、Huggingfaceからモデルをダウンロードするため、初回起動には2分程度時間がかかります。

この待ち時間を利用して、app.pyのコードを確認してみましょう。

In [12]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run 02_streamlit_app/app.py

公開URL: https://a4ba-2400-2410-2da2-2300-a5a0-6f-b1e9-469.ngrok-free.app
^C



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8504
  Network URL: http://192.168.3.14:8504

NLTK loaded successfully.
NLTK Punkt data checked/downloaded.
Database 'chat_feedback.db' initialized successfully.
NLTK Punkt data checked/downloaded.
Database 'chat_feedback.db' initialized successfully.



Fetching 2 files: 100%|██████████| 2/2 [11:51<00:00, 355.52s/it]

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 47.77it/s]
Device set to use cuda


アプリケーションの機能としては、チャット機能や履歴閲覧があります。

これらの機能を実現するためには、StreamlitによるUI部分だけではなく、SQLiteを使用したチャット履歴の保存やLLMのモデルを呼び出した推論などの処理を組み合わせることで実現しています。

- **`app.py`**: アプリケーションのエントリーポイント。チャット機能、履歴閲覧、サンプルデータ管理のUIを提供します。
- **`ui.py`**: チャットページや履歴閲覧ページなど、アプリケーションのUIロジックを管理します。
- **`llm.py`**: LLMモデルのロードとテキスト生成を行うモジュール。
- **`database.py`**: SQLiteデータベースを使用してチャット履歴やフィードバックを保存・管理します。
- **`metrics.py`**: BLEUスコアやコサイン類似度など、回答の評価指標を計算するモジュール。
- **`data.py`**: サンプルデータの作成やデータベースの初期化を行うモジュール。
- **`config.py`**: アプリケーションの設定（モデル名やデータベースファイル名）を管理します。
- **`requirements.txt`**: このアプリケーションを実行するために必要なPythonパッケージ。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [5]:
from pyngrok import ngrok
ngrok.kill()

# 03_FastAPI

ディレクトリ「03_FastAPI」に移動します。

In [ ]:
%cd /content/lecture-ai-engineering/day1/03_FastAPI

必要なライブラリをインストールします。

In [ ]:
%%capture
!pip install -r requirements.txt

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [ ]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

アプリを起動します。

「02_streamlit_app」から続けて「03_FastAPI」を実行している場合は、モデルのダウンロードが済んでいるため、すぐにサービスが立ち上がります。

「03_FastAPI」のみを実行している場合は、初回の起動時にモデルのダウンロードが始まるので、モデルのダウンロードが終わるまで数分間待ちましょう。

In [8]:
!python 03_FastAPI/app.py

^C


モデル名を設定: google/gemma-2-2b-jpn-it
FastAPIエンドポイントを定義しました。
アクティブなngrokトンネルはありません。
ポート8501に新しいngrokトンネルを開いています...
---------------------------------------------------------------------
✅ 公開URL:   https://35d0-2400-2410-2da2-2300-a5a0-6f-b1e9-469.ngrok-free.app
📖 APIドキュメント (Swagger UI): https://35d0-2400-2410-2da2-2300-a5a0-6f-b1e9-469.ngrok-free.app/docs
---------------------------------------------------------------------
(APIクライアントやブラウザからアクセスするためにこのURLをコピーしてください)
load_model_task: モデルの読み込みを開始...
使用デバイス: cuda
モデル 'google/gemma-2-2b-jpn-it' の読み込みに成功しました
load_model_task: モデルの読み込みが完了しました。
起動時にモデルの初期化が完了しました。


c:\Users\ks130\Desktop\aie_lecture\lecture-ai-engineering\day1_local\03_FastAPI\app.py:134: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
INFO:     Started server process [35420]
INFO:     Waiting for application startup.

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 72.90it/s]
Device set to use cuda
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8501 (Press CTRL+C to quit)


FastAPIが起動すると、APIとクライアントが通信するためのURL（エンドポイント）が作られます。

URLが作られるのと合わせて、Swagger UIというWebインターフェースが作られます。

Swagger UIにアクセスすることで、APIの仕様を確認できたり、APIをテストすることができます。

Swagger UIを利用することで、APIを通してLLMを動かしてみましょう。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [9]:
from pyngrok import ngrok
ngrok.kill()